In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import h5py
import datetime as dt
import scipy.io

In [23]:
# # Removing string from numbers (M, B, ...) + Assertion
# df = pd.read_csv('number_string.csv')
# df['pop'] = df['pop'].str.strip('M')
# df['pop'] = df['pop'].astype('int')
# # verifying
# assert df['pop'].dtype == 'int'
# df['category'] = df['category'].astype('category')
# df.describe()   # the category column is removed
# df['category'].describe()

In [37]:
# # Deal with Out-of-range Data
# # Approches: 1) drop 2) setting custom min and max for columns 3) treat as missing data (later)
# # 1: Dropping:
# movies = pd.read_csv('movies.csv')
# movies2 = movies[movies['rating'] <=5]                              # Solution 1
# movies1 = movies.copy()
# movies1.drop(movies1[movies1['rating']>5].index, inplace = True)       # Solution 2
# assert movies1['rating'].max() <= 5

# # 2: Setting range:
# movies3 = movies.copy()
# movies3.loc[movies3['rating']>5, 'rating'] = 5 
# assert movies3['rating'].max() <= 5

# # Special case: dates
# # for dates to be comparable, we first need to change their format
# user_signup['subs_date'] = pd.to_datetime(user_signup['sub_date']).dt.date
# today_date = dt.date.today()
# user_signup = user_signup[user_signup['subs_date']<today_date]
# # or
# user_signup.drop(user_signup[user_signup['subs_date']>today_date].index, inplace = True) 

In [19]:
# # Example 1
# # Convert tire_sizes to integer
# ride_sharing['tire_sizes'] = ride_sharing['tire_sizes'].astype('int')
# ride_sharing.loc[ride_sharing['tire_sizes'] > 27, 'tire_sizes'] = 27

# # Reconvert tire_sizes back to categorical
# ride_sharing['tire_sizes'] = ride_sharing['tire_sizes'].astype('category')
# print(ride_sharing['tire_sizes'].describe())

# # Example 2
# # Convert ride_date to date
# ride_sharing['ride_dt'] = pd.to_datetime(ride_sharing['ride_date']).dt.date
# today = dt.date.today()
# ride_sharing.loc[ride_sharing['ride_dt'] > today, 'ride_dt'] = today
# print(ride_sharing['ride_dt'].max())

In [40]:
# # Uniqeness Condition, dealing with duplicate values
# column_name = ['film']      # It can be several column names
# duplicates = movies.duplicated(subset=column_name, keep=False)
# # keep: first (keep first), last, False (keep all)
# movies[duplicates].sort_values(by = 'film')
# movies4 = movies.copy()
# movies4.drop_duplicates(subset = column_name, inplace=True, keep='last')
# # keep works the same
# # inplace = True: drop duplicated rows directly inside dataframe without creating new obj.

# # more sophisticated method:
# summaries = {'rating':'max', 'sales':'mean'}
# movies5 = movies.groupby(by = column_name).agg(summaries).reset_index()

In [41]:
# # Example 1
# duplicates = ride_sharing.duplicated(subset = 'ride_id', keep = False)
# duplicated_rides = ride_sharing[duplicates].sort_values('ride_id')
# print(duplicated_rides[['ride_id','duration','user_birth_year']])

# # Example 2
# # Drop complete duplicates from ride_sharing
# ride_dup = ride_sharing.drop_duplicates()
# statistics = {'user_birth_year': 'min', 'duration': 'mean'}
# ride_unique = ride_dup.groupby('ride_id').agg(statistics).reset_index()

# # Find duplicated values again
# duplicates = ride_unique.duplicated(subset = 'ride_id', keep = False)
# duplicated_rides = ride_unique[duplicates == True]
# assert duplicated_rides.shape[0] == 0

In [19]:
# # Membership constraints, working with categories
# # Solutions when we face a problem:
# # 1) dropping   2) remapping    3) inferring them

# blood = pd.read_csv('blood.csv')
# categories = ['A+', 'B+', 'AB+', 'O+', 'A-', 'B-', 'AB-', 'O-']
# catgry = pd.DataFrame(categories, columns=['category'])
# print(blood['group'].unique())
# inconsistents = set(blood['group']).difference(catgry['category'])
# inconsistent_rows = blood['group'].isin(inconsistents)
# blood[~inconsistent_rows]

In [18]:
# # Example
# # Print categories DataFrame
# print(categories.info())

# # Print unique values of survey columns in airlines
# print('Cleanliness: ', airlines['cleanliness'].unique(), "\n")
# print('Safety: ', airlines['Safety'].unique(), "\n")
# print('Satisfaction: ', airlines['Satisfaction'].unique(), "\n")

# # Find the cleanliness category in airlines not in categories
# cat_clean = set(airlines['cleanliness']).difference(categories['cleanliness'])

# # Find rows with that category
# cat_clean_rows = airlines['cleanliness'].isin(cat_clean)

# # Print rows with inconsistent category
# print(airlines[cat_clean_rows])

# # Print rows with consistent categories only
# print(airlines[~cat_clean_rows])

In [81]:
# # Working with Categories - continue

# # Value Inconsistency, 
# # upper lower strings case + spaces
# marrge = pd.read_csv('Marriage.csv')
# marrge_status = marrge['marrge_st']
# count = marrge_status.value_counts()
# print(count)
# marrge['marrge_st'] = marrge['marrge_st'].str.strip()   # remove spaces
# marrge['marrge_st'] = marrge['marrge_st'].str.upper()
# count = marrge_status.value_counts()
# print(count)

# # creating category based on money
# group_name = ['A', 'B', 'C']
# marrge['wealth_ctgry'] = pd.qcut(marrge['money'], q=3, labels=group_name)
# marrge[['money','wealth_ctgry']]
# # instead:
# ranges = [0,10,40,np.inf]
# marrge['wealth_ctgry'] = pd.cut(marrge['money'], bins=ranges, labels=group_name)
# marrge[['money','wealth_ctgry']]

# # Collapse Catehories
# A = [['A','BBB'],['B','BB'],['C','BBBB'],['D','AA'],['E','AAAA']]
# A_d=pd.DataFrame(A, columns=['name','value'])
# mapping = {'BBB':'B', 'BB':'B', 'BBBB':'B', 'BBc':'B', 'AA':'A', 'AAAA':'A'}
# A_d['value'] = A_d['value'].replace(mapping)
# print(A_d['value'].unique())

In [82]:
# # Example 1
# # Print unique values of both columns
# print(airlines['dest_region'].unique())
# print(airlines['dest_size'].unique())

# # Lower dest_region column and then replace "eur" with "europe"
# airlines['dest_region'] = airlines['dest_region'].str.lower() 
# airlines['dest_region'] = airlines['dest_region'].replace({'eur':'europe'})

# # Remove white spaces from `dest_size`
# airlines['dest_size'] = airlines['dest_size'].str.strip()

# # Verify changes have been effected
# print(airlines['dest_size'].unique())
# print(airlines['dest_region'].unique())

# # Example 2
# label_ranges = [0, 60, 180, np.inf]
# label_names = ['short', 'medium', 'long']
# airlines['wait_type'] = pd.cut(airlines['wait_min'], bins = label_ranges, 
#                                 labels = label_names)

# mappings = {'Monday':'weekday', 'Tuesday':'weekday', 'Wednesday': 'weekday', 
#             'Thursday': 'weekday', 'Friday': 'weekday', 
#             'Saturday': 'weekend', 'Sunday': 'weekend'}

# airlines['day_week'] = airlines['day'].replace(mappings)

In [83]:
# # Cleaning Text Data
# # Working and preparing phone numbers texts
# phones['phone number'] = phones['phone number'].str.replace("+","00")
# phones['phone number'] = phones['phone number'].str.replace("-","")
# digits = phones['phone number'].str.len()
# phones.loc[digits < 10, 'phone number'] = np.nan
# sanity_check = phones['phone number'].str.len()
# assert sanity_check.min() >= 10
# # check numbers do not have "+" or "-"
# assert phones['phone number'].str.contains("+|-").any() ==  False

# # regular expressions 
# phones['phone number'] = phones['phone number'].str.replace(r"\D+","")


In [84]:
# # Example
# airlines['full_name'] = airlines['full_name'].str.replace("Dr.","")
# airlines['full_name'] = airlines['full_name'].str.replace("Mr.","")
# airlines['full_name'] = airlines['full_name'].str.replace("Miss","")
# airlines['full_name'] = airlines['full_name'].str.replace("Ms.","")
# assert airlines['full_name'].str.contains('Ms.|Mr.|Miss|Dr.').any() == False

# # Example 2
# resp_length = airlines['survey_response'].str.len()
# airlines_survey = airlines[resp_length > 40]
# assert airlines_survey['survey_response'].str.len().min() > 40
# print(airlines_survey['survey_response'])


In [86]:
# # Uniformity
# # First we can plot a scatter to find outliers
# plt.scatter(x = 'Date', y = 'temperature', data = Temperatures)
# # For example, by plotting the temperatures, we find out that three points are too high and their unit should be ferenhite.
# temp_frnh = Temperatures.loc[Temperatures['temperature']>40,'temperature']
# temp_celc = (temp_frnh - 32) / (5/9)
# Temperatures.loc[Temperatures['temperature']>40,'temperature'] = temp_celc
# assert Temperatures['temperature'].max() < 40

# # Different Types of Birthday Formats
# # most dates are recognizable by datetime format (pd.to_datetime())
# births['birthday'] = pd.to_datetime(births['birthday'])   # This way we will face errors (ValueError) probably
# births['birthday'] = pd.to_datetime(births['birthday'],
#                                     # Attempt to infer the format of each date
#                                     infer_date_time_format = True,
#                                     # Return NA where conversion failed
#                                     errors = 'coerce')

# # for setting a specific format
# births['birthday'] = births['birthday'].dt.strftime("%d-%m-%y")

# # Generally, it is essential to take into account the source of data before starting to clean it  

In [87]:
# # Example 1
# acct_eu = banking['acct_cur'] == 'euro'
# banking.loc[acct_eu, 'acct_amount'] = banking.loc[acct_eu, 'acct_amount'] * 1.1
# banking.loc[acct_eu, 'acct_cur'] = 'dollar'
# assert banking['acct_cur'].unique() == 'dollar'

# # Example 2
# # Print the header of account_opend
# print(banking['account_opened'].head())

# # Convert account_opened to datetime
# banking['account_opened'] = pd.to_datetime(banking['account_opened'],
#                                            # Infer datetime format
#                                            infer_datetime_format = True,
#                                            # Return missing value for error
#                                            errors = 'coerce') 

# # Get year of account opened
# banking['acct_year'] = banking['account_opened'].dt.strftime('%Y')

# # Print acct_year
# print(banking['acct_year'])

In [14]:
# # Cross field validation: using multiple fields in a detaset to sanity check the integrity of them
# integ = pd.read_csv('Integrity.csv')
# sum_classes = integ[['seatA','seatB','seatC']].sum(axis=1)
# check_eq = sum_classes == integ['Total']
# inconsistent_seats = integ[~check_eq]
# consistent_seats = integ[check_eq]

# # Another example: The code does not work
# users['birthday'] = pd.to_datetime(users['birthday'])
# today = dt.date.today()
# age_manual = today.year - users['birthday'].dt.year
# age_eq = age_manual == users['Age']
# inconsistent_age = users[~age_eq]

In [13]:
# # Example 
# fund_columns = ['fund_A', 'fund_B', 'fund_C', 'fund_D']
# inv_equ = banking[fund_columns].sum(axis = 1) == banking['inv_amount']
# consistent_inv = banking[inv_equ]
# inconsistent_inv = banking[~inv_equ]
# print("Number of inconsistent investments: ", inconsistent_inv.shape[0])

In [33]:
# # Completeness, dealing with missing data
# # Missing Types: 
# # Missing completely @ random: no systematic relationship between missing data and other values
# # Missing random 
# # Missing not @ random:There is a systematic relationship between missing data and observed ones
# # Approaches to deal with them:
# # Drop / impute with statiscal measures like mean, median, ... / 
# # impute an algorithm approach / machine learning approach

# information = pd.read_csv('missing.csv')
# print(information.isna().sum())        # .sum() provides a summary
# missing = information['pop'].isna() | information['money'].isna()
# missed_data = information[missing]
# complete_data = information[~missing]
# complete_data.describe()

# info2 = information.dropna(subset=['pop'])
# # OR
# pop_mean = information['pop'].mean()
# info3 = information.fillna({'pop':pop_mean})


# # for visualizing the missing data:
# import missingno as msno
# msno.matrix(information)
# plt.show()

# sorted_pop = information.sort_values(by = 'pop', ascending=True)
# msno.matrix(sorted_pop)
# plt.show()


In [32]:
# # Example
# print(banking.isna().sum())
# msno.matrix(banking)
# plt.show()
# missing_investors = banking[banking['inv_amount'].isna()]
# investors = banking[~banking['inv_amount'].isna()]
# banking_sorted = banking.sort_values(by = 'age')
# msno.matrix(banking_sorted)
# plt.show()

# # Example 2
# # Drop missing values of cust_id
# banking_fullid = banking.dropna(subset = ['cust_id'])

# # Compute estimated acct_amount
# acct_imp = banking_fullid['inv_amount'] * 5

# # Impute missing acct_amount with corresponding acct_imp
# banking_imputed = banking_fullid.fillna({'acct_amount':acct_imp})

# # Print number of missing values
# print(banking_imputed.isna().sum())

In [12]:
# # Minimum editing distance: a systematic way to realize how close two strings are.
# # The # of operation(insert, delete, change the position, ...) needed to convert a string to another.
# # There are several packages for doing so such as fuzz
# from thefuzz import fuzz
# from thefuzz import process
# print(fuzz.WRatio('Reeding','Reading'))        # 0: not similar   100: the same
# print(fuzz.WRatio('Houston Rockets','Rockets'))
# print(fuzz.WRatio("Houston Rockets vs Los Angeles Lakers", 'Rockets vs Lakers'))
# string1 = "Houston Rockets vs Los Angeles Lakers"
# choices = pd.Series(['Rockets vs Lakers','Lakers vs Rockets','Houson vs Los Angeles','Heat vs Bulls'])
# print(process.extract(string1, choices, limit = choices.shape[0]))    # For comparing a string with an array of strings

# # Now we want to categorize a list based on the similarity of its elements
# categories = pd.DataFrame(['New York', 'California'], columns=['state'])
# survey = pd.read_csv('state.csv')
# for state in categories['state']:
#     matches = process.extract(state, survey['state'], limit=survey.shape[0])
#     for potential_matches in matches:
#         if potential_matches[1] >= 80:
#             survey.loc[survey['state'] == potential_matches[0], 'state'] = state

# print(survey)            

In [13]:
# # Example 1
# from thefuzz import process
# unique_types = restaurants['cuisine_type'].unique()

# # Calculate similarity of 'asian', 'american', and 'italian' to all values of unique_types
# print(process.extract('asian', unique_types, limit = len(unique_types)))
# print(process.extract('american', unique_types, limit = len(unique_types)))
# print(process.extract('italian', unique_types, limit = len(unique_types)))

# # Example 2
# # Iterate through categories
# for cuisine in categories:  
#   # Create a list of matches, comparing cuisine with the cuisine_type column
#   matches = process.extract(cuisine, restaurants['cuisine_type'], limit=len(restaurants.cuisine_type))

#   # Iterate through the list of matches
#   for match in matches:
#      # Check whether the similarity score is greater than or equal to 80
#     if match[1] >= 80:
#       # If it is, select all rows where the cuisine_type is spelled this way, and set them to the correct cuisine
#       restaurants.loc[restaurants['cuisine_type'] == match[0]] = cuisine
      
# # Inspect the final result
# print(restaurants['cuisine_type'].unique())



In [16]:
# # Record Linkage. The code does not work
# # Here we have two dataframes and we want to find pairs between them
# # works well for data with different formats

# import recordlinkage
# indexer = recordlinkage.Index()
# indexer.block('state')
# pairs = indexer.index(census_A, census_B)

# compare_cl = recordlinkage.Compare()

# compare_cl.exact('date_of_birth','date_of_birth', label = 'date_of_birth')
# compare_cl.exact('state','state', label = 'state')

# compare_cl.string('surname','surname',threshold = 0.85, label = 'surname')
# compare_cl.string('address_1','address_1',threshold = 0.85, label = 'address_1')

# potential_matches = compare_cl.compute(pairs, census_A, census_B)
# matches = potential_matches[potential_matches.sum(axis = 1) >= 3]


In [17]:
# # Example
# # In the last lesson, you cleaned the restaurants dataset to make it ready for building a restaurants recommendation engine. 
# # You have a new DataFrame named restaurants_new with new restaurants to train your model on, that's been scraped from a new data source.
# # You've already cleaned the cuisine_type and city columns using the techniques learned throughout the course. 
# # However you saw duplicates with typos in restaurants names that require record linkage instead of joins with restaurants.
# # In this exercise, you will perform the first step in record linkage and generate possible pairs of 
# # rows between restaurants and restaurants_new. 
# # Both DataFrames, pandas and recordlinkage are in your environment.

# # Create an indexer and object and find possible pairs
# indexer = recordlinkage.Index()

# # Block pairing on cuisine_type
# indexer.block('cuisine_type')

# # Generate pairs
# pairs = indexer.index(restaurants, restaurants_new)

# # In the last exercise, you generated pairs between restaurants and restaurants_new
# # in an effort to cleanly merge both DataFrames using record linkage.
# # When performing record linkage, there are different types of matching you can perform between different 
# # columns of your DataFrames, including exact matches, string similarities, and more.
# # Now that your pairs have been generated and stored in pairs, you will find exact matches in the city and 
# # cuisine_type columns between each pair, and similar strings for each pair in the rest_name column. 
# # Both DataFrames, pandas and recordlinkage are in your environment.

# # Create a comparison object
# comp_cl = recordlinkage.Compare()

# # Find exact matches on city, cuisine_types - 
# comp_cl.exact('city', 'city', label='city')
# comp_cl.exact('cuisine_type', 'cuisine_type', label='cuisine_type')

# # Find similar matches of rest_name
# comp_cl.string('rest_name', 'rest_name', label='name', threshold = 0.8) 

# # Get potential matches and print
# potential_matches = comp_cl.compute(pairs, restaurants, restaurants_new)
# print(potential_matches)

In [ ]:
# # The rest of previous block:
# duplicate_rows = matches.index.get_level_values(1)
# census_B_dupl = census_B[census_B.index.isin(duplicate_rows)]
# census_B_new = census_B[~census_B.index.isin(duplicate_rows)]
# full_census = census_A.append(census_B_new)

In [ ]:
# # Example - continued
# # In the last lesson, you've finished the bulk of the work on your effort to link restaurants and restaurants_new. 
# # You've generated the different pairs of potentially matching rows, searched for exact matches between the cuisine_type and city columns, 
# # but compared for similar strings in the rest_name column. You stored the DataFrame containing the scores in potential_matches.
# # Now it's finally time to link both DataFrames. 
# # You will do so by first extracting all row indices of restaurants_new that are matching across the 
# # columns mentioned above from potential_matches. 
# # Then you will subset restaurants_new on these indices, then append the non-duplicate values to restaurants. 
# # All DataFrames are in your environment, alongside pandas imported as pd.

# # Isolate potential matches with row sum >=3
# matches = potential_matches[potential_matches.sum(axis = 1) >= 3]

# # Get values of second column index of matches
# matching_indices = matches.index.get_level_values(1)

# # Subset restaurants_new based on non-duplicate values
# non_dup = restaurants_new[~restaurants_new.index.isin(matching_indices)]

# # Append non_dup to restaurants
# full_restaurants = restaurants.append(non_dup)
# print(full_restaurants)